<a href="https://colab.research.google.com/github/YousefAbua/Intro-To-ML/blob/main/Homework6/NeuralNetworkQ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
import pandas as pd
from sklearn import preprocessing

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import and read file
file_path = '/content/drive/My Drive/Intro to ML/Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))

varlist = ['area','bedrooms','bathrooms','stories','parking', 'price']

charlist = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea']

def binary_map(x):
  return x.map({'yes' : 1, 'no' : 0})

# Applying binary map
housing[charlist] = housing[charlist].apply(binary_map)

scaler = preprocessing.MinMaxScaler()
housing[varlist] = scaler.fit_transform(housing[varlist])

housing.pop('furnishingstatus')
x = housing.values
y = housing.pop('price')

In [4]:
x = torch.tensor(x)
y = torch.tensor(y)

In [5]:
n_samples = x.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [6]:
x_train = x[train_indices]
y_train = y[train_indices]

x_test = x[val_indices]
y_test = y[val_indices]

In [7]:
def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_test, y_train, y_test):
  for epoch in range(1, n_epochs + 1):
    predict_train = model(x_train)
    loss_train = loss_fn(predict_train, y_train)

    predict_test = model(x_test)
    loss_test = loss_fn(predict_test, y_test)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch % 500 == 0 :
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}," f" Validation loss {loss_test.item():.4f}")


In [8]:
def loss_fn(predict, y):
  squared_diffs = (predict - y)**2
  return squared_diffs.mean()

In [11]:
# Model with 1 hidden layer
seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12, 32)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(32, 1))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=12, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=32, out_features=1, bias=True)
)

In [12]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    x_train = x_train.to(torch.float32),
    x_test = x_test.to(torch.float32),
    y_train = y_train.to(torch.float32),
    y_test = y_test.to(torch.float32))

Epoch 500, Training loss 0.0274, Validation loss 0.0283
Epoch 1000, Training loss 0.0265, Validation loss 0.0273
Epoch 1500, Training loss 0.0263, Validation loss 0.0271
Epoch 2000, Training loss 0.0262, Validation loss 0.0270
Epoch 2500, Training loss 0.0262, Validation loss 0.0269
Epoch 3000, Training loss 0.0262, Validation loss 0.0269


In [15]:
# Model with 3 hidden layers
seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12, 32)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear1', nn.Linear(32, 64)),
    ('hidden_activation1', nn.Tanh()),
    ('hidden_linear2', nn.Linear(64, 16)),
    ('hidden_activation2', nn.Tanh()),
    ('output_linear', nn.Linear(16, 1))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=12, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (hidden_linear1): Linear(in_features=32, out_features=64, bias=True)
  (hidden_activation1): Tanh()
  (hidden_linear2): Linear(in_features=64, out_features=16, bias=True)
  (hidden_activation2): Tanh()
  (output_linear): Linear(in_features=16, out_features=1, bias=True)
)

In [18]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    x_train = x_train.to(torch.float32),
    x_test = x_test.to(torch.float32),
    y_train = y_train.to(torch.float32),
    y_test = y_test.to(torch.float32))

Epoch 500, Training loss 0.0261, Validation loss 0.0268
Epoch 1000, Training loss 0.0261, Validation loss 0.0268
Epoch 1500, Training loss 0.0261, Validation loss 0.0268
Epoch 2000, Training loss 0.0261, Validation loss 0.0268
Epoch 2500, Training loss 0.0261, Validation loss 0.0268
Epoch 3000, Training loss 0.0261, Validation loss 0.0268
